# MAS DSE 200: Homework 2 - Pandas

#### Tasks:

- Submission on Gradescope:
  - Submit this Jupyter notebook to "Homework 2"


---

Remember: when in doubt, read the documentation first.

Python - https://docs.python.org/3/

NumPy - https://numpy.org/doc/stable/

pandas - https://pandas.pydata.org/docs/

## Instructions

* You don’t need to explain your approach (unless specified) so please be concise in your submission.
* To obtain full marks for a question, both the answer and the code should be correct.
* Completely wrong (or missing) code with correct answer will result in zero marks.
* Please code the solution in the space provided.

### Imports

Import necessary packages

In [1]:
!pip install pandas numpy requests pillow rasterio

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 22.3/22.3 MB 46.2 MB/s eta 0:00:00


In [2]:
import pandas as pd
import numpy as np
import requests
from pathlib import Path
from PIL import Image
from io import BytesIO
import rasterio
from rasterio.transform import from_bounds

## Part 1: Titanic

### Preliminaries

* Grab the dataset from `https://raw.githubusercontent.com/justmarkham/DAT8/master/data/titanic.csv` and store it in a pandas dataframe called `passengers`.

In [3]:
# YOUR CODE HERE
# download from https://raw.githubusercontent.com/justmarkham/DAT8/master/data/titanic.csv
with open('titanic.csv', 'wb') as f:
    response = requests.get('https://raw.githubusercontent.com/justmarkham/DAT8/master/data/titanic.csv')
    f.write(response.content)


In [4]:
# open the csv file as a pandas dataframe
passengers = pd.read_csv('https://raw.githubusercontent.com/justmarkham/DAT8/master/data/titanic.csv')

### 1: Get to know your data - **20 points**

**1.1**: Print the first 15 entries in the dataframe to see what the columns are and what some values will look like - **5 points**

In [5]:
# YOUR CODE HERE
print(passengers.shape)
print(passengers.head(16))

(891, 12)
    PassengerId  Survived  Pclass  \
0             1         0       3   
1             2         1       1   
2             3         1       3   
3             4         1       1   
4             5         0       3   
5             6         0       3   
6             7         0       1   
7             8         0       3   
8             9         1       3   
9            10         1       2   
10           11         1       3   
11           12         1       1   
12           13         0       3   
13           14         0       3   
14           15         0       3   
15           16         1       2   

                                                 Name     Sex   Age  SibSp  \
0                             Braund, Mr. Owen Harris    male  22.0      1   
1   Cumings, Mrs. John Bradley (Florence Briggs Th...  female  38.0      1   
2                              Heikkinen, Miss. Laina  female  26.0      0   
3        Futrelle, Mrs. Jacques Heath (Lily May 

**1.2**: Next, set the index of the dataframe to the `PassengerId` column, and print the first 10 elements again to ensure the change took place - **5 points**

In [6]:
# YOUR CODE HERE
passengers = passengers.set_index('PassengerId')
print(passengers.head(10))

             Survived  Pclass  \
PassengerId                     
1                   0       3   
2                   1       1   
3                   1       3   
4                   1       1   
5                   0       3   
6                   0       3   
7                   0       1   
8                   0       3   
9                   1       3   
10                  1       2   

                                                          Name     Sex   Age  \
PassengerId                                                                    
1                                      Braund, Mr. Owen Harris    male  22.0   
2            Cumings, Mrs. John Bradley (Florence Briggs Th...  female  38.0   
3                                       Heikkinen, Miss. Laina  female  26.0   
4                 Futrelle, Mrs. Jacques Heath (Lily May Peel)  female  35.0   
5                                     Allen, Mr. William Henry    male  35.0   
6                                          

**1.3**: How many samples are there in this dataset? - **5 points**

In [7]:
# YOUR CODE HERE
sample_size = passengers.shape[0]
print(f"There are {sample_size} samples in the Titanic dataset.")

There are 891 samples in the Titanic dataset.


**1.4** How many samples contain `null`/`NaN` in atleast one of the columns? - **5 points**

In [8]:
# YOUR CODE HERE
contains_null = passengers.isnull().sum()
print(contains_null.sum())

866


### 2: Summary statistics - **30 points**

**2.1**: Print the `min`, `max`, `mean` and `median` of age and fare of all passengers - **10 points**

Hint - Look at [`DataFrame.agg`](https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.agg.html#pandas.DataFrame.agg)

In [9]:
# YOUR CODE HERE
# print(passengers["Age"].min())
# youngest_person = passengers[passengers["Age"] == passengers["Age"].min()]
# oldest_person = passengers[passengers["Age"] == passengers["Age"].max()]
# print(youngest_person)
# print(oldest_person)
print(passengers[['Age', 'Fare']].agg(['min', 'max', 'mean', 'median']))

              Age        Fare
min      0.420000    0.000000
max     80.000000  512.329200
mean    29.699118   32.204208
median  28.000000   14.454200


**2.2**: What is the average ticket fare price for male vs female passengers on the Titanic? - **10 points**

Note - The output should only have `Sex` and `Fare`

*Hint* - Look at [`DataFrame.groupby`](https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.groupby.html#pandas.DataFrame.groupby)

In [10]:
# YOUR CODE HERE
avg_fare = passengers.groupby("Sex")["Fare"].mean().reset_index()
print(avg_fare.to_string(index=False))

   Sex      Fare
female 44.479818
  male 25.523893


**2.3** What is the mean age for each of the sex and cabin class combinations? - **10 points**

In [11]:
# YOUR CODE HERE
avg_age_sex = passengers.groupby("Sex")["Age"].mean().reset_index()
print(avg_age_sex)
avg_age_cabin = passengers.groupby("Cabin")["Age"].mean().fillna("Age Unknown").reset_index()
print(avg_age_cabin)

      Sex        Age
0  female  27.915709
1    male  30.726645
    Cabin          Age
0     A10         36.0
1     A14  Age Unknown
2     A16         48.0
3     A19  Age Unknown
4     A20         49.0
..    ...          ...
142   F33         29.0
143   F38  Age Unknown
144    F4          2.5
145    G6        14.75
146     T         45.0

[147 rows x 2 columns]


### 3:  Number of passengers in different classes - **20 points**

**3.1**: What is the number of passenges in different classes according to this dataset? (Hint: Pclass represents the class of a passenger.) - **5 points**

In [12]:
# YOUR CODE HERE
class_count = passengers["Pclass"].value_counts()
print(class_count)

Pclass
3    491
1    216
2    184
Name: count, dtype: int64


**3.2** How many passengers in 1st class (`Pclass = 1`) are women (`Sex = female`) above the age of 27? - **5 points**

In [13]:
# YOUR CODE HERE
female_passengers = passengers[(passengers["Pclass"] == 1) & (passengers["Sex"] == "female") & (passengers["Age"] > 27)]
len(female_passengers)

56

**3.3** What fraction of passengers from each class survived? (`Survived=1`) - **10 points**

In [14]:
# YOUR CODE HERE
survival_fraction = passengers.groupby("Pclass")["Survived"].mean()
print(survival_fraction)

Pclass
1    0.629630
2    0.472826
3    0.242363
Name: Survived, dtype: float64


### 4:  Fares - **30 points**

**4.1**: How many different fares were charged on the Titanic based on the dataset? - **5 points**

In [15]:
# YOUR CODE HERE
len(passengers["Fare"].unique())

248

**4.2**: Find the top 10 fares charged from the passengers. **Report these fare values**, and then **calculate the total number of passengers** who paid one of these top 10 fare amounts - **10 points**

In [16]:
# YOUR CODE HERE
top_10_fares = passengers["Fare"].value_counts().head(10)
print(top_10_fares)

print(top_10_fares.sum())



Fare
8.0500     43
13.0000    42
7.8958     38
7.7500     34
26.0000    31
10.5000    24
7.9250     18
7.7750     16
7.2292     15
26.5500    15
Name: count, dtype: int64
276


**4.3**: Create a new dataset, called `passengers_filtered`, that includes only entries of passengers who paid one of these top 10 fares. **Report the number of samples** in the original dataset and in the new dataset to ensure the desired effect took place - **10 points**

**hint:** Check out the Pandas Series function [isin](https://pandas.pydata.org/pandas-docs/stable/generated/pandas.Series.isin.html)

In [31]:
# YOUR CODE HERE
passengers_filetered = passengers[passengers["Fare"].isin(top_10_fares.index)]
print(f"{len(passengers)} samples in the original dataset")
print(f"{len(passengers_filetered)} samples in the new dataset")

891 samples in the original dataset
276 samples in the new dataset


### 5:  Ages - **30 points**

**5.1**: What was the minimum, maximum and average age of passengers on the Titanic? - **5 points**

In [18]:
# YOUR CODE HERE
#passengers["Age"].describe()
print(passengers["Age"].min())
print(int(passengers["Age"].max()))
average_age = passengers["Age"].mean()
print(f"{average_age:.2f}")


0.42
80
29.70


**5.2**: How many passengers on the Titanic were within one standard deviation of the mean age calculated in **5.1**? - **10 points**

In [19]:
# YOUR CODE HERE
std_deviation_age= passengers["Age"].std()

min_std_age = average_age - std_deviation_age
max_std_age = average_age + std_deviation_age

passengers_within_std = passengers[(passengers["Age"] >= min_std_age) & (passengers["Age"] <= max_std_age)]
print(len(passengers_within_std))


516


**5.3**: How many of the passengers found in **5.2** were females over the age of 25? - **5 points**

In [20]:
# YOUR CODE HERE
females_within_std_over_25 = passengers_within_std[(passengers_within_std["Sex"] == "female") & (passengers_within_std["Age"] > 25)]
print(len(females_within_std_over_25))

103


**5.4**: What are the 10 **most** common ages of passengers according to this dataset? - **5 points**

In [21]:
# YOUR CODE HERE
top_10_ages = passengers["Age"].value_counts().head(10)
print(top_10_ages)

Age
24.0    30
22.0    27
18.0    26
28.0    25
30.0    25
19.0    25
21.0    24
25.0    23
36.0    22
29.0    20
Name: count, dtype: int64


## Part 2: Beer Review - 20 points

Use the `beer_reviews` dataframe created for you

In [25]:
reviews = []
response = requests.get("https://dse200.dev/Day2/beer_50000.json")
for line in response.text.splitlines():
    reviews.append(eval(line))

beer_reviews = pd.DataFrame(reviews)

**6.1**: Which are the top 15 beers with the highest average ratings (`review/overall`)? - **10 points**

In [52]:
# YOUR CODE HERE
print(len(beer_reviews[(beer_reviews["review/overall"]==5.0)]))

top_beers = beer_reviews.sort_values(by="review/overall", ascending=False).head(15)
print(top_beers[["beer/name","review/overall"]])
# print(top_beers)


3517
                            beer/name  review/overall
18033      Founders Backwoods Bastard             5.0
21431        Founders Breakfast Stout             5.0
47886  Stoudt's Blonde Double MaiBock             5.0
38276               Dank Imperial Red             5.0
21432        Founders Breakfast Stout             5.0
18057      Founders Backwoods Bastard             5.0
21435        Founders Breakfast Stout             5.0
3185           Barrel Aged Naked Evil             5.0
3162           Barrel Aged Naked Evil             5.0
3165           Barrel Aged Naked Evil             5.0
12874            Mathias Imperial IPA             5.0
12873            Mathias Imperial IPA             5.0
3195           Barrel Aged Naked Evil             5.0
21468        Founders Breakfast Stout             5.0
48591                    Stoudts Pils             5.0


**6.2**: Which of the following - `review/palate`, `review/taste`, `review/aroma`, length of `review/text`(number of words in the review text) - correlate highest with `review/overall`? - **10 points**

NOTE - `review/text` is of type string while the other reviews are of type float. Use the length of `review/text` instead. You may need to create a new column in the data frame

In [ ]:
# YOUR CODE HERE
review_length= len(beer_reviews["review/text"])

## Part 3: Geospatial Data - 10 points

In [53]:

data_dir = Path('fire_data')
data_dir.mkdir(exist_ok=True)

In [55]:
# Select a new area
# http://bboxfinder.com/#-117.500249,32.665157,-116.820470,33.012173
# Go to the above link and select an area of interest
# Example bbox for San Diego area: [-117.500249, 32.665157, -116.820470, 33.012173]
bbox = [34.817734,-1.657331,35.169296,-1.292157]  # Replace with your chosen area

print(f"Study Area Bounding Box: {bbox}")
print(f"Longitude: {bbox[0]:.2f} to {bbox[2]:.2f}")
print(f"Latitude: {bbox[1]:.2f} to {bbox[3]:.2f}")

Study Area Bounding Box: [34.817734, -1.657331, 35.169296, -1.292157]
Longitude: 34.82 to 35.17
Latitude: -1.66 to -1.29


In [56]:
def get_naip_imagery(bbox, output_path):
    """
    Fetch NAIP imagery - simple version
    Always gets 1024x1024 pixels, no calculations
    """
    base_url = "https://imagery.nationalmap.gov/arcgis/rest/services/USGSNAIPImagery/ImageServer/exportImage"

    params = {
        'bbox': f"{bbox[0]},{bbox[1]},{bbox[2]},{bbox[3]}",
        'bboxSR': '4326',
        'size': '1024, 1024',
        'imageSR': '4326',
        'format': 'tiff',  # Get TIFF to preserve all bands
        'pixelType': 'U16',
        'f': 'image'
    }

    print(f"Requesting NAIP imagery: 1024x1024 pixels")
    response = requests.get(base_url, params=params, timeout=120)

    if response.status_code == 200:
        # Save directly - no conversion needed
        with open(output_path, 'wb') as f:
            f.write(response.content)

        print(f"✓ NAIP imagery saved to {output_path}")
        return True
    else:
        print(f"Error fetching NAIP: {response.status_code}")
        return False


# Use it
naip_file = data_dir / 'naip_imagery.tif'
print("Downloading NAIP imagery...")
success = get_naip_imagery(bbox, naip_file)

if success:
    # Check what we got
    with rasterio.open(naip_file) as src:
        print(f"Bands: {src.count}")
        print(f"Size: {src.width}x{src.height}")

Requesting NAIP imagery: 1024x1024 pixels
✓ NAIP imagery saved to fire_data/naip_imagery.tif
Bands: 4
Size: 1024x1024


### Task - Geospatial Data Analysis - 10 points

Based on the NAIP imagery data you've downloaded, create your own analytics to explore the geospatial data.

**Requirements (10 points total):**
1. **Visualize the imagery** (3 points) - Display the RGB bands properly
2. **Identify color patterns** (3 points) - Create masks to identify 2 colors (e.g., green for vegetation, brown for bare soil) or any other colors of your choice
3. **Analyze spatial patterns** (4 points) - Use your color masks to calculate statistics (e.g., percentage of green vs brown areas, spatial distribution)

You should create 3 different visualizations or analyses demonstrating your understanding of geospatial data manipulation with pandas and rasterio.

In [57]:
# Your code here
fig, ax = plt.subplots(figsize=(10, 10))

NameError: name 'plt' is not defined